### 리뷰 데이터를 이용한 연관 단어 추출기 만들기 (with 엘라스틱 서치)

In [2]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import requests
import json
import gradio as gr

In [3]:
review_df = pd.read_csv('datasets/review/review.csv')
review_df.head()

,review_no,accom_no,user_no,accom_nm,content,review_rate1,review_rate2,review_rate3
0,43f61fa6,9b0057ac,b9677a1e,세인트존스 호텔,지은지 얼마안되서 그런지 실내 인테리어도 좋았고 침대시트 등 모든것이 만족하였다. ...,100,100,100
1,694f3882,9b0057ac,956bc8bc,세인트존스 호텔,1. 시티뷰를 결제했는데 오션뷰를 주더라구요 한달 전에 예약한 거여서 몰랐는데 나중...,100,100,100
2,c6756abd,9b0057ac,41376307,세인트존스 호텔,객실도 넓고 좋았어요~~~,100,100,100
3,7b98fef9,9b0057ac,0b4c9081,세인트존스 호텔,일단 체크인 줄 굉장히 깁니다. 입실시간 최소 30분 전에 줄 서야 4시 겨우 넘어...,100,100,40
4,58f50b9e,9b0057ac,2fc2aed9,세인트존스 호텔,늦은휴가를위해 검색도중 인피니티풀이 핫하다는 세인트존스호텔이 인기가많길래 결정했습...,100,100,100


In [4]:
# 검색엔진 접속 테스트
url = 'http://elasticsearch:9200'
headers = {'Content-Type': 'application/json'}
response = requests.get(url, headers=headers).json()
response


{'name': '78bd77f38fca',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'XA45LTSCTKGcfxeSqaAOuA',
 'version': {'number': '7.14.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'dd5a0a2acaa2045ff9624f3729fc8a6f40835aa1',
  'build_date': '2021-07-29T20:49:32.864135063Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [5]:
# 엘라스틱서치 토크나이저 함수
def extract_words_from_text(text):
    # Elasticsearch의 endpoint 설정
    url = 'http://elasticsearch:9200/_analyze'
    
    # 요청할 데이터 준비: 사용자 정의 분석기 사용 시 'analyzer' 설정을 변경
    headers = {'Content-Type': 'application/json'}
    payload = {
        "analyzer": "nori",  # 'standard', 'nori' 등의 분석기 지정 가능
        "text": text
    }
    
    # POST 요청 수행
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    # 응답 데이터 처리
    if response.status_code == 200:
        tokens = []
        for token in response.json()['tokens']:
            if token['token'].isdigit() :
                continue
            if len(token['token']) < 2 :
                continue
            if isinstance(token['token'], int):
                continue
            tokens.append(token['token'])
        return tokens

    else:
        print("Error:", response.status_code, response.text)
        return []


In [6]:
# 단어 추출 테스트
extract_words_from_text("일단 체크인 줄 굉장히 깁니다. 입실시간 최소 30분 전에 줄 서야 4시 겨우 넘어..")

['체크', '입실', '시간', '최소']

In [7]:
# 데이터셋 단어 추출
review_df['nouns'] = review_df.content.apply(extract_words_from_text)
review_df.head()

,review_no,accom_no,user_no,accom_nm,content,review_rate1,review_rate2,review_rate3,nouns
0,43f61fa6,9b0057ac,b9677a1e,세인트존스 호텔,지은지 얼마안되서 그런지 실내 인테리어도 좋았고 침대시트 등 모든것이 만족하였다. ...,100,100,100,"[얼마, 그렇, 실내, 인테리어, 침대, 시트, 만족, 위치, 산책, 직원, 부분,..."
1,694f3882,9b0057ac,956bc8bc,세인트존스 호텔,1. 시티뷰를 결제했는데 오션뷰를 주더라구요 한달 전에 예약한 거여서 몰랐는데 나중...,100,100,100,"[시티, 결제, 오션, 예약, 모르, 나중, 확인, 그것, 행운, 오션, 아니, 여..."
2,c6756abd,9b0057ac,41376307,세인트존스 호텔,객실도 넓고 좋았어요~~~,100,100,100,[객실]
3,7b98fef9,9b0057ac,0b4c9081,세인트존스 호텔,일단 체크인 줄 굉장히 깁니다. 입실시간 최소 30분 전에 줄 서야 4시 겨우 넘어...,100,100,40,"[체크, 입실, 시간, 최소, 체크인, 체크인, 감상, 위하, 테라스, 나가, 투숙..."
4,58f50b9e,9b0057ac,2fc2aed9,세인트존스 호텔,늦은휴가를위해 검색도중 인피니티풀이 핫하다는 세인트존스호텔이 인기가많길래 결정했습...,100,100,100,"[휴가, 위하, 검색, 도중, 인피니티, 세인트존스, 호텔, 인기, 결정, 사람, ..."


In [8]:
# 데이터 준비
dataset = review_df.nouns.tolist()
nouns_list = []
for i in dataset:
    nouns_list.extend(i)
nouns_list = list(set(nouns_list))
len(nouns_list)

5804

In [9]:
# 숫자 제거
review_df.nouns = review_df.nouns.apply(lambda x : [noun for noun in x if not noun.isdigit()])
nouns_list = [noun for noun in nouns_list if not noun.isdigit()]

In [10]:
# 모델 준비
dataset = review_df.nouns.tolist()
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
as_df = pd.DataFrame(te_ary, columns=te.columns_)

In [11]:
# 연관 규칙 생성
frequent_itemsets = apriori(
        as_df,
        min_support=0.0045,      
        use_colnames=True,
        low_memory=True,
        max_len=3              
    )
rules = association_rules(frequent_itemsets, metric="conviction", min_threshold=0.001)


In [12]:
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ㅜㅜ),(객실),0.015443,0.181508,0.004613,0.298701,1.645663,1.0,0.001810,1.167109,0.398496,0.023983,0.143182,0.162058
1,(객실),(ㅜㅜ),0.181508,0.015443,0.004613,0.025414,1.645663,1.0,0.001810,1.010231,0.479348,0.023983,0.010128,0.162058
2,(ㅜㅜ),(오션),0.015443,0.168672,0.005215,0.337662,2.001884,1.0,0.002610,1.255142,0.508321,0.029148,0.203277,0.184289
3,(오션),(ㅜㅜ),0.168672,0.015443,0.005215,0.030916,2.001884,1.0,0.002610,1.015966,0.602014,0.029148,0.015715,0.184289
4,(ㅜㅜ),(체크인),0.015443,0.199960,0.005215,0.337662,1.688650,1.0,0.002127,1.207904,0.414208,0.024809,0.172119,0.181870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71583,"(편하, 해변)",(호텔),0.013037,0.282792,0.007020,0.538462,1.904092,1.0,0.003333,1.553951,0.481087,0.024306,0.356479,0.281642
71584,"(호텔, 해변)",(편하),0.033694,0.084035,0.007020,0.208333,2.479117,1.0,0.004188,1.157008,0.617435,0.063406,0.135702,0.145933
71585,(편하),"(호텔, 해변)",0.084035,0.033694,0.007020,0.083532,2.479117,1.0,0.004188,1.054380,0.651369,0.063406,0.051576,0.145933
71586,(호텔),"(편하, 해변)",0.282792,0.013037,0.007020,0.024823,1.904092,1.0,0.003333,1.012086,0.662033,0.024306,0.011942,0.281642


In [16]:
def recommend_products(antecedent, rules_df, metric='confidence', top_n=10):
    """
    주어진 상품(antecedent)에 대해 연관 규칙을 기반으로 추천 상품 리스트를 반환하는 함수.
    
    Parameters:
    antecedent (str): 추천의 기준이 되는 상품.
    rules_df (DataFrame): 연관 규칙이 담긴 데이터프레임.
    metric (str): 정렬 기준 
    top_n (int): 반환할 추천 상품의 최대 개수.
    
    Returns:
    list의 개별 요소 n개: 추천 상품 리스트.
    """
    # 주어진 상품에 대한 연관 규칙 필터링
    filtered_rules = rules_df[rules_df['antecedents'].apply(lambda x: antecedent in x)]
    
    # 신뢰도(confidence)가 높은 순으로 정렬
    sorted_rules = filtered_rules.sort_values(by=metric, ascending=False)
    
    # 상위 N개의 결과에서 추천 상품(consequents) 추출
    recommendations = [""] * 10  # 항상 10개의 빈 문자열로 초기화
    
    # 실제 추천 결과가 있는 경우에만 값을 채움
    if not sorted_rules.empty:
        cal_recommendations = sorted_rules['consequents'].head(top_n).apply(lambda x: list(x)[0]).tolist()
        for i, rec in enumerate(cal_recommendations):
            if i < 10:  # 10개까지만 처리
                recommendations[i] = rec
    
    # 항상 10개의 값을 반환 (빈 값은 ""로 패딩됨)
    return recommendations[0], recommendations[1], recommendations[2], recommendations[3], recommendations[4], recommendations[5], recommendations[6], recommendations[7], recommendations[8], recommendations[9]


In [17]:
# 테스트
recommend_products(antecedent='오션', rules_df=rules, metric='confidence', top_n=5)

('이베이', '스카', '엘레', '시설', '해변', '', '', '', '', '')

In [ ]:
# Gradio 인터페이스 설정
interface = gr.Interface(
    fn=lambda product_name: recommend_products(product_name, rules, metric='lift', top_n=10),
    inputs=gr.Dropdown(choices=nouns_list, label="단어 선택"),
    outputs=[gr.Textbox(label="연관 단어 리스트" + str(i)) for i in range(1, 11)],
    title="리뷰 기반 연관 단어 추출",
    description="태깅용으로 활용 가능"
)

# 인터페이스 실행, 화면 높이 조정
interface.launch(height=1200, server_name="0.0.0.0")


In [236]:
interface.close()

Closing server running on port: 7861
